## Plot Obs From SynopticData

### Object Oriented Approach

In [ ]:
import urllib.request as req
import os.path
import json
from helpers import mapper, asos_list, logo
from datetime import datetime
import re
import numpy as np
import pandas as pd
from metpy.plots import StationPlot, StationPlotLayout, sky_cover, current_weather, wx_code_map
from metpy.calc import wind_components
from metpy.units import units
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

class SurfaceMap:
    
    # Set up query parameters
    mapExtent = [-97,-77,32.5,44.5]
    stnDensity = 1.2 # Larger number is more sparse
    startTime = datetime(2019,10,25,23)
    endTime = datetime(2019,10,26,12)
    interval = '1H' # Interval to 
    time_format = 'hidden' # 'hidden' or 'normal
    caseName = 'IND'
    productShortName = 'sfcobs'
    imgParentDirectory = '/coda/s2/CLIMATE/www/html/reforecast/IND/data/obs/Surface/11Z20991024/ohio/'
    
    window = '60' # Window of time to look for obs before ob time, in minutes
    API_ROOT = "https://api.synopticdata.com/v2/"
    API_TOKEN = "9450ad26ab0445029f5911023c2506e8"
    units = 'temp|F,speed|kts,pres|mb,height|ft,precip|in,alti|pa' # Get units in correct form
    
    
    def generate_plots(self):
        datelist = self.dt_range()
        fNumber = 1
        for time in datelist:
            self.fNumber = fNumber
            self.plot_data(time)
            fNumber = fNumber + 1
            
    
    def plot_data(self, time):
        ''' Plot data on the map '''
        data = self.process_data(time)
        ax = mapper(self.mapExtent,(18,14.4),proj='Miller', resolution='10m', counties=True)

        # Custom layout to format temp and dewpoint
        custom_layout = StationPlotLayout()
        custom_layout.add_value('NW', 'temp', fmt='.0f', units='degF', color='darkred')
        custom_layout.add_value('SW', 'dew', fmt='.0f', units='degF', color='darkgreen')

        # Station plot
        stationplot = StationPlot(ax, np.array(data['lon']), np.array(data['lat']), clip_on=True, transform=ccrs.PlateCarree(), fontsize=10)

        u, v = wind_components(np.array(data['wind']) * units('knots'), np.array(data['dir']) * units.degree)
        stationplot.plot_barb(u,v, sizes={'emptybarb':0}, linewidth=0.5)
        stationplot.plot_symbol('C', data['cloud'], sky_cover)
        stationplot.plot_symbol('W', data['wx'], current_weather, fontsize=12)
        stationplot.plot_parameter('NE', data['slp'], formatter=lambda v: format(10 * v, '.0f')[-3:], fontweight='bold')
        stationplot.plot_text((2,-0.5), data['stid'])
        custom_layout.plot(stationplot, data)

        # Add logo
        logo(ax)
        
        # Save figure
        self.save_fig(time)
    
    def process_data(self, time):
        ''' Process the data returned by the query '''
        stnData = self.get_data(time)
        data = []
        for stn in stnData:
            stid = stn['STID']
            lat = float(stn['LATITUDE'])
            lon = float(stn['LONGITUDE'])
            try:
                metar = stn['OBSERVATIONS']['metar_value_1']['value']
            except KeyError:
                metar = None

            # Match and assign cloud cover code
            try:
                match = re.findall(self.cloud_re, metar)
                cloud_cover = max(np.array([self.get_cloud_cover(cloud) for cloud in match]))
                cloud_cover = int(cloud_cover*8.)
            except:
                cloud_cover = 10 # Not reported

            # Match and assign present weather code
            try:
                match = re.findall(self.wx_re, metar)
                wx = match[0] # Only consider first wx condition
                pres_wx = wx_code_map[str(wx)]
            except:
                pres_wx = 0 # Not reported

            # Get temp, dewpoint, wind, direction, slp
            try:
                temp = float(stn['OBSERVATIONS']['air_temp_value_1']['value'])
            except KeyError:
                temp = None
            try:
                dew = float(stn['OBSERVATIONS']['dew_point_temperature_value_1']['value'])
            except KeyError:
                dew = None
            try:
                wind = float(stn['OBSERVATIONS']['wind_speed_value_1']['value'])
            except KeyError:
                wind = None
            try:
                direction = float(stn['OBSERVATIONS']['wind_direction_value_1']['value'])
            except KeyError:
                direction = None
            try:
                slp = float(stn['OBSERVATIONS']['sea_level_pressure_value_1']['value'])
            except KeyError:
                slp = None

            # Populate dictionary and append to data list
            stndata={'stid':stid,'lat':lat,'lon':lon,'temp':temp, 'dew':dew, 'slp':slp, 'wind':wind, 'dir':direction, 'cloud':cloud_cover, 'wx':pres_wx, 'metar':metar}
            data.append(stndata)
            
        # Convert to format metpy can handle
        data = pd.DataFrame(data).to_dict('list')
        
        return data
    
        
    def get_data(self, time):
        ''' Gets station data based on query parameters'''
        api_request_url = os.path.join(self.API_ROOT, "stations/nearesttime")
        stations = self.query_asos()
        time = datetime.strftime(time, '%Y%m%d%H%M')
        api_request_url += "?token={}&stid={}&attime={}&within={}&units={}".format(self.API_TOKEN, stations, time, self.window, self.units)
        response = req.urlopen(api_request_url)
        api_text_data = response.read()
        use_data = json.loads(api_text_data)
        stnData = use_data['STATION']
        
        return stnData
    
    
    def save_fig(self, time):
        ''' Setting time format of title and saving figure '''
        if self.time_format == 'hidden':
            plt.title('Surface Observations', fontsize=13, fontweight='bold', loc='left')
            plt.title(r"$\bf{Valid:\/\/}$"+ datetime.strftime(time, '%HZ %a %b DD YYYY'), fontsize=13, loc='right')
        else:
            plt.title('Surface Observations', fontsize=13, fontweight='bold', loc='left')
            plt.title(r"$\bf{Valid:\/\/}$"+ datetime.strftime(time, '%HZ %a %d %b %Y'), fontsize=13, loc='right')

        imgName = self.productShortName+'_'+self.caseName+'_f'+str(self.fNumber)+'.png'
        plt.savefig(self.imgParentDirectory+imgName, bbox_inches='tight')
        plt.close()
    
    
    def dt_range(self):
        ''' List of dates between 2 datetime objects as specified by interval '''
        datelist = pd.date_range(start=self.startTime,end=self.endTime,freq=self.interval).to_pydatetime()
        
        return datelist
        
        
    def query_asos(self):
        ''' Return list of ASOS stations to query synopticdata database with '''
        stations = asos_list(self.mapExtent, point_density=self.stnDensity)
        stations = [stn['id'] for stn in stations]
        stations = ",".join(stations)
        
        return stations
    
    def get_cloud_cover(self, code):
        if 'OVC' in code:
            return 1.0
        elif 'VV' in code:
            return 1.0
        elif 'BKN' in code:
            return 6.0/8.0
        elif 'SCT' in code:
            return 4.0/8.0
        elif 'FEW' in code:
            return 2.0/8.0
        else:
            return 0
    
    # Regex for metar parsing
    cloud_re = re.compile('VV|FEW|SCT|SKC|CLR|BKN|OVC')
    wx_re = re.compile('(TSNO|VA|FU|HZ|DU|BLDU|SA|BLSA|VCBLSAVCBLDU|BLPY|PO|VCPO|VCDS|VCSS|BR|BCBR|BC|MIFG|VCTS|VIRGA \
                           |VCSH|TS|THDR|VCTSHZ|TSFZFG|TSBR|TSDZ|SQ|FC|\+FC|DS|SS|DRSA|DRDU|TSUP|\+DS|\+SS|-BLSN|BLSN|\+BLSN| \
                           VCBLSN|DRSN|\+DRSN|VCFG|BCFG|PRFG|FG|FZFG|-VCTSDZ|-DZ|-DZBR|VCTSDZ|DZ|\+VCTSDZ|\+DZ|-FZDZ|-FZDZSN| \
                           FZDZ|\+FZDZ|FZDZSN|-DZRA|DZRA|\+DZRA|-VCTSRA|-RA|-RABR|VCTSRA|RA|RABR|RAFG|\+VCTSRA|\+RA|-FZRA| \
                           -FZRASN|-FZRABR|-FZRAPL|-FZRASNPL|TSFZRAPL|-TSFZRA|FZRA|\+FZRA|FZRASN|TSFZRA|-DZSN|-RASN|-SNRA|-SNDZ| \
                           RASN|\+RASN|SNRA|DZSN|SNDZ|\+DZSN|\+SNDZ|-VCTSSN|-SN|-SNBR|VCTSSN|SN|\+VCTSSN|\+SN|VCTSUP|IN|-UP|UP| \
                           \+UP|-SNSG|SG|-SG|IC|-FZDZPL|-FZDZPLSN|FZDZPL|-FZRAPLSN|FZRAPL|\+FZRAPL|-RAPL|-RASNPL|-RAPLSN| \
                           \+RAPL|RAPL|-SNPL|SNPL|-PL|PL|-PLSN|-PLRA|PLRA|-PLDZ|\+PL|PLSN|PLUP|\+PLSN|-SH|-SHRA|SH|SHRA|\+SH| \
                           \+SHRA|-SHRASN|-SHSNRA|\+SHRABR|SHRASN|\+SHRASN|SHSNRA|\+SHSNRA|-SHSN|SHSN|\+SHSN|-GS|-SHGS|FZRAPLGS| \
                           -SNGS|GSPLSN|GSPL|PLGSSN|GS|SHGS|\+GS|\+SHGS|-GR|-SHGR|-SNGR|GR|SHGR|\+GR|\+SHGR|-TSRA|TSRA|TSSN| \
                           TSPL|-TSDZ|-TSSN|-TSPL|TSPLSN|TSSNPL|-TSSNPL|TSRAGS|TSGS|TSGR|\+TSRA|\+TSSN|\+TSPL|\+TSPLSN|TSSA| \
                           TSDS|TSDU|\+TSGS|\+TSGR)')

In [ ]:
# Make the plots
stations = SurfaceMap()
stations.generate_plots()